In [1]:
import plyvel
import dbobjects_pb2 as KaspadDB
from utils import *

In [8]:
sep = b'/'
level = (0).to_bytes(1, 'little')  # Default level for stores with block-level
ghostdag_data_store = b'block-ghostdag-data'
header_store = b'block-headers'
header_count_key = b'block-headers-count'
relations_store = b'block-relations'
candidate_pruning_point_key = b'candidate-pruning-point-hash'
pruning_block_index_key = b'pruning-block-index'
pruning_by_index_store = b'pruning-point-by-index'

In [3]:
db_path = r'D:\kaspad-data\datadir2-cp-23.12T00.30'
db = plyvel.DB(db_path)
prefix = db.get(b'active-prefix')

In [4]:
# def hash_str(db_hash):
#     return encode_hash(db_hash.hash)
# KaspadDB.DbHash.__str__ = hash_str
# KaspadDB.DbHash.__repr__ = hash_str

In [5]:
candidate_bytes = db.get(prefix + sep + candidate_pruning_point_key)
cpp = KaspadDB.DbHash()
cpp.ParseFromString(candidate_bytes)
cpp

hash: "\367\220\226[\377\013D\020\214(\343\037\332q\361#jum!\252\373v\345U\002n\212\3479W\000"

In [6]:
ghostdag_data_bucket = prefix + sep + level + sep + ghostdag_data_store + sep
ghostdag_data_bytes = db.get(ghostdag_data_bucket + cpp.hash)
gdd = KaspadDB.DbBlockGhostdagData()
gdd.ParseFromString(ghostdag_data_bytes)
gdd

blueScore: 2104897
blueWork: "\004\006:\032\331\246\272\275"
selectedParent {
  hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
}
mergeSetBlues {
  hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
}
mergeSetBlues {
  hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
}
bluesAnticoneSizes {
  blueHash {
    hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
  }
  anticoneSize: 1
}
bluesAnticoneSizes {
  blueHash {
    hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
  }
  anticoneSize: 1
}

In [22]:
pp_index_bytes = db.get(prefix + sep + pruning_block_index_key)
pp_index = int.from_bytes(pp_index_bytes, 'little')
pp_bytes = db.get(prefix + sep + pruning_by_index_store + sep + 
                  pp_index.to_bytes(8, 'big'))
pp = KaspadDB.DbHash()
pp.ParseFromString(pp_bytes)
pp

hash: "-2$\203b\3263z\300\301S\207\032-L\275\243r4f\232(\340\256_s\235,\226\225K\366"

In [23]:
db.close()